In [3]:
# basic information about the program
import transformers
print(transformers.__version__)
import torch
print(torch.__version__)

print(torch.cuda.is_available())  
print(torch.cuda.device_count())  

4.51.3
2.5.1
True
1


In [4]:
import os
import torch

#agent
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.json import OutputParserException
from langchain.prompts import ChatPromptTemplate
from typing import TypedDict
from langgraph.graph import MessagesState
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from typing import Literal

from transformers import (AutoTokenizer, 
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          pipeline,
                          TextStreamer)
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
import pandas as pd

import socket
from concurrent.futures import ThreadPoolExecutor

In [3]:
from function.generation import init_model, get_model_response

# initialize the model 
llm = init_model()
output = get_model_response(user_prompt="hello",
                            system_prompt= None,
                            llm=llm
                            )
print(output)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


<|begin_of_text|>
        <|begin_of_text|>
        <|start_header_id|>user<|end_header_id|>
        hello
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        

c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 hello<|eot_id|>
hello


In [9]:
from function.agent import build_agent, Agent_State
from tqdm import tqdm
import re 
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, AIMessage
import socket


graph = build_agent()


messages_state = MessagesState(messages=[HumanMessage(content="the initial message are not used as the first message only use number & topic & tld, but i need to initialize the object")])

number = 20
topic = "beatles"
tld = ".com"
agent_state: Agent_State = {
    "state": messages_state,
    "number": number,
    "topic": topic,
    "tld": tld,
    "domain": {},
    "llm": llm
}
output_graph = graph.invoke(agent_state)
domain_list = output_graph['domain']['domain']

print("check whether the domain is registered:\n")
# verify which domain is not registered

def is_valid_domain_format(domain):
    # example.com / www.google.com / my-site.org
    pattern = r'^(?!-)[A-Za-z0-9-]{1,63}(?<!-)(\.[A-Za-z]{2,})+$'
    return re.match(pattern, domain) is not None

# Function to check if a domain is registered
def is_domain_registered(domain):
    domain = domain.strip().lower()
    if not is_valid_domain_format(domain):
        return True
    try:
        socket.gethostbyname(domain)
        return True
    except socket.gaierror:
        return False
        
# Use ThreadPoolExecutor to check domains concurrently
unregistered_domains = []

for domain in tqdm(domain_list):
    if not is_domain_registered(domain):
        unregistered_domains.append(domain)
print("\nUnregistered domains:")
unregistered_domains

---Generating domain---
<|begin_of_text|>
        <|begin_of_text|>
        <|start_header_id|>system<|end_header_id|>
        You are required to generate domain names based on a given topic. For every request, you will be provided with a topic, and your task is to generate domain names that include the word 'DOMAIN' before each suggested name. Below are examples of how the input request and your response should look like:
Example:
Request: Please help me generate 3 domain names about "cooking" with tld.com
Response:
DOMAIN: TastyHub.com
DOMAIN: RecipeRush.com
DOMAIN: TheCookingCoach.com
        <|eot_id|>
        <|start_header_id|>user<|end_header_id|>
        Please help me generate 20 domains related to "beatles" with tld ".com" and please do not generate domain that already exisit
        <|eot_id|>
        <|start_header_id|>assistant<|end_header_id|>
        

c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Here are 20 domain name suggestions related to "beatles" with the.com extension:

1. DOMAIN: FabFourBeatles.com
2. DOMAIN: LennonLegacy.com
3. DOMAIN: TheBeatlemania.com
4. DOMAIN: SgtPeppersDomain.com
5. DOMAIN: MagicalMysteryTour.com
6. DOMAIN: YellowSubmarineDomain.com
7. DOMAIN: AbbeyRoadDomain.com
8. DOMAIN: TheBeatlemania.com
9. DOMAIN: JohnLennonLegacy.com
10. DOMAIN: PaulMcCartneyDomain.com
11. DOMAIN: GeorgeHarrisonHub.com
12. DOMAIN: Ringo StarrDomain.com
13. DOMAIN: TheBeatlesOfficial.com
14. DOMAIN: BeatlesFanClub.com
15. DOMAIN: LetItBeDomain.com
16. DOMAIN: AllYouNeedIsLove.com
17. DOMAIN: StrawberryFieldDomain.com
18. DOMAIN: PennyLaneDomain.com
19. DOMAIN: TheBeatlesClassics.com
20. DOMAIN: ComeTogetherDomain.com<|eot_id|>
new_messages: [HumanMessage(content='the initial message are not used as the first message only use number & topic & tld, but i need to initialize the object', additional_kwargs={}, response_metadata={}), 'Here are 20 domain name suggestions related 

c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 Reasoning: There are multiple domains which fit the format of a standard domain inside the text.
Answer: TRUE<|eot_id|>
Now we try to parse the output to structured format and inspect the result.

Prompt Template:
 For the given text, extract the following information:

- **domain**: Identify all domain names mentioned in the text and return them as a list. 

**Text To Analyze:**
{text}
**End Of Text**
here is a example output:
```
{{
	"domain": string  //     Extract the domain names mentioned in the text.
    For example, if the text contains "The domains are hello.com and byebye.net",
    the output should be a list: ["hello.com", "byebye.net"].
    
}}
```
Please output the result strictly in valid JSON format, without any additional text, commentary, or explanation.


Formatted prompt:
 For the given text, extract the following information:

- **domain**: Identify all domain names mentioned in the text and return them as a list. 

**Text To Analyze:**
Here are 20 domain name sugg

c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\aa\anaconda3\envs\llm_agent\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 {
  "domain": [
    "FabFourBeatles.com",
    "LennonLegacy.com",
    "TheBeatlemania.com",
    "SgtPeppersDomain.com",
    "MagicalMysteryTour.com",
    "YellowSubmarineDomain.com",
    "AbbeyRoadDomain.com",
    "TheBeatlemania.com",
    "JohnLennonLegacy.com",
    "PaulMcCartneyDomain.com",
    "GeorgeHarrisonHub.com",
    "Ringo StarrDomain.com",
    "TheBeatlesOfficial.com",
    "BeatlesFanClub.com",
    "LetItBeDomain.com",
    "AllYouNeedIsLove.com",
    "StrawberryFieldDomain.com",
    "PennyLaneDomain.com",
    "TheBeatlesClassics.com",
    "ComeTogetherDomain.com"
  ]
}<|eot_id|>
Raw output from model:
 {
  "domain": [
    "FabFourBeatles.com",
    "LennonLegacy.com",
    "TheBeatlemania.com",
    "SgtPeppersDomain.com",
    "MagicalMysteryTour.com",
    "YellowSubmarineDomain.com",
    "AbbeyRoadDomain.com",
    "TheBeatlemania.com",
    "JohnLennonLegacy.com",
    "PaulMcCartneyDomain.com",
    "GeorgeHarrisonHub.com",
    "Ringo StarrDomain.com",
    "TheBeatlesOfficial.c

100%|██████████| 20/20 [02:39<00:00,  7.95s/it]


Unregistered domains:


['fabfourbeatles.com',
 'lennonlegacy.com',
 'thebeatlemania.com',
 'sgtpeppersdomain.com',
 'yellowsubmarinedomain.com',
 'abbeyroaddomain.com',
 'thebeatlemania.com',
 'johnlennonlegacy.com',
 'paulmccartneydomain.com',
 'georgeharrisonhub.com',
 'beatlesfanclub.com',
 'letitbedomain.com',
 'strawberryfielddomain.com',
 'pennylanedomain.com',
 'thebeatlesclassics.com',
 'cometogetherdomain.com']